In [15]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import numpy as np

import qcodes as qc

qc.set_mp_method('spawn')  # force Windows behavior on mac

# this makes a widget in the corner of the window to show and control
# subprocesses and any output they would print to the terminal
qc.show_subprocess_widget()

In [16]:
# spawn doesn't like function or class definitions in the interpreter
# session - had to move them to a file.
from toymodel import AModel, MockGates, MockSource, MockMeter, AverageGetter, AverageAndRaw

# now create this "experiment"
model = AModel()
gates = MockGates('gates', model=model)
source = MockSource('source', model=model)
meter = MockMeter('meter', model=model)

station = qc.Station(gates, source, meter)

# could measure any number of things by adding arguments to this
# function call, but here we're just measuring one, the meter amplitude
station.set_measurement(meter.amplitude)

# it's nice to have the key parameters be part of the global namespace
# that way they're objects that we can easily set, get, and slice
# this could be simplified to a station method that gathers all parameters
# and adds them all as (disambiguated) globals, printing what it did
# something like:
#   station.gather_parameters(globals())
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude

# once we have implemented a monitor, defining a station will start a
# DataServer process, and you would see it in the subprocess widget,
# or via active_children() as here:
# qc.active_children()

In [17]:
meter.amplitude()

0.117

In [18]:
vsd(0.5)
vsd()

0.5

In [19]:
# we can get the measured quantities right now
station.measure()

[0.627]

In [20]:
# start a Loop (which by default runs in a seprarate process)
# the sweep values are defined by slicing the parameter object
# but more complicated sweeps (eg nonlinear, or adaptive) can
# easily be used instead

# Notice that I'm using an explicit location and `overwrite=True` here so that
# running this notebook over and over won't result in extra files.
# But if you leave these out, you get a new timestamped DataSet each time.
# data = qc.Loop(c0[-20:20:0.1], 0.03).run(location='testsweep', overwrite=True, background=False)

data = qc.Loop(c0[-20:20:0.1], 0.03).run_temp()#(location='testsweep', overwrite=True, background=False)

# now there should be two extra processes running, DataServer and a sweep
# I'll omit the active_children call now because you can see them in the
# subprocess widget

In [21]:
# manually bring the data into the main process and display it as numbers
data.sync()
data.arrays

{'amplitude': DataArray[400]: amplitude
 array([ 0.627,  0.661,  0.69 ,  0.711,  0.723,  0.723,  0.711,  0.687,
         0.653,  0.613,  0.569,  0.524,  0.479,  0.437,  0.398,  0.362,
         0.329,  0.3  ,  0.273,  0.25 ,  0.229,  0.21 ,  0.194,  0.179,
         0.166,  0.154,  0.143,  0.134,  0.125,  0.117,  0.11 ,  0.103,
         0.098,  0.092,  0.087,  0.083,  0.078,  0.075,  0.071,  0.068,
         0.065,  0.062,  0.059,  0.057,  0.054,  0.052,  0.05 ,  0.048,
         0.046,  0.045,  0.043,  0.045,  0.046,  0.048,  0.05 ,  0.052,
         0.054,  0.057,  0.059,  0.062,  0.065,  0.068,  0.071,  0.075,
         0.078,  0.083,  0.087,  0.092,  0.098,  0.103,  0.11 ,  0.117,
         0.125,  0.134,  0.143,  0.154,  0.166,  0.179,  0.194,  0.21 ,
         0.229,  0.25 ,  0.273,  0.3  ,  0.329,  0.362,  0.398,  0.437,
         0.479,  0.524,  0.569,  0.613,  0.653,  0.687,  0.711,  0.723,
         0.723,  0.711,  0.69 ,  0.661,  0.627,  0.661,  0.69 ,  0.711,
         0.723,  0.723, 

In [22]:
# live-updating plot, that syncs the data and stops updating when it's finished
# plot = qc.MatPlot(data.amplitude)
plotQ = qc.QtPlot(data.amplitude)

TypeError: sequence item 0: expected str instance, bool found

In [23]:
data2 = qc.Loop(c1[-15:15:1], 0.1).loop(c0[-15:12:.5], 0.01).each(
    meter.amplitude, # first measurement, at c2=0 -> amplitude_0 bcs it's action 0
    qc.Task(c2.set, 1), # action 1 -> c2.set(1)
    qc.Wait(0.001),
    meter.amplitude, # second measurement, at c2=1 -> amplitude_4 bcs it's action 4
    qc.Task(c2.set, 0)
    ).run(location='test2d', overwrite=True)

# use the subplot and add features of qc.MatPlot
# plot2 = qc.MatPlot(data2.amplitude_0, cmap=plt.cm.hot, figsize=(12, 4.5), subplots=(1, 2))
# plot2.add(data2.amplitude_3, cmap=plt.cm.hot, subplot=2)

# the equivalent in QtPlot
plot2Q = qc.QtPlot(data2.amplitude_0, figsize=(1200, 500))
plot2Q.add(data2.amplitude_3, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='test2d'
   amplitude_3: amplitude
   amplitude_0: amplitude
   chan1: chan1
   chan0: chan0
started at 2016-04-18 16:23:14


In [24]:
data3 = qc.Loop(c1[-15:15:1], 0.1).each(
    qc.Task(c0.set, -10),
    qc.Task(c2.set, 0),
    # a 1D measurement
    meter.amplitude,
    # a 2D sweep, .each is actually unnecessary bcs this is the default measurement
    qc.Loop(c0[-15:15:1], 0.001).each(meter.amplitude),
    qc.Task(c0.set, -10),
    # a 2D sweep with the same outer but different inner loop
    qc.Loop(c2[-10:10:0.2], 0.001),
    AverageGetter(meter.amplitude, c2[-10:10:0.2], 0.001)
).run(location='test_multi_d', overwrite=True)

# several plots updating simultaneously
# plot3 = qc.MatPlot(data3.amplitude_3_0, cmap=plt.cm.hot)
# plot3b = qc.MatPlot(data3.amplitude_5_0, cmap=plt.cm.hot, figsize=(12, 4.5), subplots=(1,2))
# plot3b.add(data3.avg_amplitude, subplot=2)
plot3Q = qc.QtPlot(data3.amplitude_3_0)
plot3bQ = qc.QtPlot(data3.amplitude_5_0, figsize=(1200, 500))
plot3bQ.add(data3.avg_amplitude, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='test_multi_d'
   amplitude_3_0: amplitude
   avg_amplitude: avg_amplitude
   chan2: chan2
   chan1: chan1
   amplitude_5_0: amplitude
   chan0: chan0
   amplitude_2: amplitude
started at 2016-04-18 16:23:45


In [26]:
# An example of a parameter that returns several values of different dimension
# This produces the last two arrays from data3, but only takes the data once.
data4 = qc.Loop(c1[-15:15:1], 0.1).each(
    AverageAndRaw(meter.amplitude, c2[-10:10:0.2], 0.001)
).run(location='test_complex_param', overwrite=True)

# plot4 = qc.MatPlot(data4.amplitude, cmap=plt.cm.hot, subplots=(1,2), figsize=(12, 4.5))
# plot4.add(data4.avg_amplitude, subplot=2)
plot4Q = qc.QtPlot(data4.amplitude, figsize=(1200, 500))
plot4Q.add(data4.avg_amplitude, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='test_complex_param'
   avg_amplitude: avg_amplitude
   chan2: chan2
   chan1: chan1
   amplitude: amplitude
started at 2016-04-18 16:24:13
